In [ ]:
!pip install pandas tqdm

In [ ]:
from pathlib import Path
import pandas as pd
from tqdm import tqdm
import re


def extract_hash(string):
    match = re.search(r"FEATS_(.*?)_SWAPS", string)
    return match.group(1) if match else None

In [ ]:
!beaker dataset fetch 01J686ACE7H0MH097V0KJKR7W1

In [ ]:
# FEATURES_PATH = Path("../../data/all_features_helpsteer2/data")
FEATURES_PATH = Path("data")
CSV_PATH = Path("test.csv")

In [ ]:
filepaths = list(FEATURES_PATH.glob("*.jsonl"))
num_swaps = {}
for fp in tqdm(filepaths):
    df = pd.read_json(fp, lines=True)
    try:
        num_swaps[fp.name] = df["is_swapped"].value_counts()[True]
    except Exception:
        num_swaps[fp.name] = len(df)

swaps_df = (
    pd.DataFrame([num_swaps])
    .transpose()
    .rename(columns={0: "num_swaps"})
    .reset_index()
    .rename(columns={"index": "filename"})
)

In [ ]:
swaps_df["hash"] = swaps_df["filename"].apply(extract_hash)

In [ ]:
swaps_df

In [ ]:
result_df = pd.read_csv(CSV_PATH)
combined = result_df.merge(swaps_df, how="left", on="hash")

In [ ]:
combined

In [ ]:
combined.to_csv("combined.csv")